In [1]:
import qlib
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
# 修正导入路径：从qlib.backtest导入回测相关功能
from qlib.backtest import backtest as normal_backtest
from qlib.contrib.evaluate import risk_analysis
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

# 初始化QLib（注意：provider_uri需要指向你的数据目录）
# 如果你之前的数据路径是C:\Users\Administrator\.qlib\qlib_data\cn_data
# 可以改为：provider_uri=r'C:\Users\Administrator\.qlib\qlib_data\cn_data'
qlib.init(provider_uri=r'C:\Users\Administrator\.qlib\qlib_data\cn_data', region=REG_CN)
    

#### Do not import qlib package in the repository directory in case of importing qlib from . without compiling #####
ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
#### Do not import qlib package in the repository directory in case of importing qlib from . without compiling #####
ModuleNotFoundError. DEnsembleModel and LGBModel are skipped. (optional: maybe installing lightgbm can fix it.)
#### Do not import qlib package in the repository directory in case of importing qlib from . without compiling #####
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).
#### Do not import qlib package in the repository directory in case of importing qlib from . without compiling #####
ModuleNotFoundError. LinearModel is skipped(optional: maybe installing scipy and sklearn can fix it).
#### Do not import qlib package in the repository directory in case of importing qlib from . without compiling #####
ModuleNot

ModuleNotFoundError: No module named 'qlib.data._libs.rolling'

In [4]:
market = "csi300"
benchmark = "SH000300"
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}
# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

[40568:MainThread](2025-07-22 16:53:14,577) INFO - qlib.timer - [log.py:127] - Time cost: 76.130s | Loading data Done
[40568:MainThread](2025-07-22 16:53:15,077) INFO - qlib.timer - [log.py:127] - Time cost: 0.174s | DropnaLabel Done
[40568:MainThread](2025-07-22 16:53:16,613) INFO - qlib.timer - [log.py:127] - Time cost: 1.535s | CSZScoreNorm Done
[40568:MainThread](2025-07-22 16:53:16,629) INFO - qlib.timer - [log.py:127] - Time cost: 2.050s | fit & process data Done
[40568:MainThread](2025-07-22 16:53:16,630) INFO - qlib.timer - [log.py:127] - Time cost: 78.182s | Init data Done


In [10]:
import qlib
import mlflow  # 直接使用MLflow API
from mlflow.entities import Experiment  # MLflow实验类
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.utils import init_instance_by_config
from qlib.workflow import R

# 初始化QLib
qlib.init(
    provider_uri=r'C:\Users\Administrator\.qlib\qlib_data\cn_data',
    region=REG_CN
)

# 配置参数（与之前一致）
market = "csi300"
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {"class": "Alpha158", "module_path": "qlib.contrib.data.handler", "kwargs": data_handler_config},
            "segments": {"train": ("2008-01-01", "2014-12-31"), "valid": ("2015-01-01", "2016-12-31"), "test": ("2017-01-01", "2020-08-01")},
        },
    },
}

# 初始化模型和数据集
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# --------------------------
# 关键：用MLflow直接创建实验
# --------------------------
# 实验名称和存储路径（解决参数缺失问题）
experiment_name = "train_model"
artifact_location = "mlruns/train_model"  # 模型存储路径

# 直接创建新实验（避免读取旧实验的问题）
exp_id = mlflow.create_experiment(
    name=experiment_name,
    artifact_location=artifact_location  # 显式传递两个必要参数
)


# 启动MLflow运行（替代QLib的R.start()）
with mlflow.start_run(experiment_id=exp_id) as run:
    # 将QLib的记录器绑定到MLflow运行
    R.set_experiment_id(exp_id)
    R.set_recorder_id(run.info.run_id)
    
    # 训练模型
    model.fit(dataset)
    
    # 保存模型到MLflow（即artifact_location路径）
    mlflow.pyfunc.log_model(
        artifact_path="trained_model",
        python_model=model  # 保存QLib的LGBModel
    )
    
    # 输出关键信息
    print(f"训练完成！")
    print(f"实验ID: {exp_id}")
    print(f"记录器ID: {run.info.run_id}")
    print(f"模型存储路径: {artifact_location}/{run.info.run_id}/artifacts/trained_model/")

[40568:MainThread](2025-07-22 17:10:44,102) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[40568:MainThread](2025-07-22 17:10:44,105) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[40568:MainThread](2025-07-22 17:10:44,106) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/Administrator/.qlib/qlib_data/cn_data')}
[40568:MainThread](2025-07-22 17:11:59,788) INFO - qlib.timer - [log.py:127] - Time cost: 75.681s | Loading data Done
[40568:MainThread](2025-07-22 17:12:00,229) INFO - qlib.timer - [log.py:127] - Time cost: 0.152s | DropnaLabel Done
[40568:MainThread](2025-07-22 17:12:01,754) INFO - qlib.timer - [log.py:127] - Time cost: 1.524s | CSZScoreNorm Done
[40568:MainThread](2025-07-22 17:12:01,769) INFO - qlib.timer - [log.py:127] - Time cost: 1.978s | fit & process data Done
[40568:MainThread](2025-07-22 17:12:01,771) INFO - qlib.timer - [log.py:127] - Ti

TypeError: Experiment.__init__() missing 2 required positional arguments: 'name' and 'artifact_location'

In [14]:
import qlib
from qlib.config import REG_CN
from qlib.utils import init_instance_by_config
from qlib.workflow import R
import mlflow

# 初始化QLib
qlib.init(
    provider_uri=r'C:\Users\Administrator\.qlib\qlib_data\cn_data',
    region=REG_CN
)

# 配置参数
market = "csi300"
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {"class": "Alpha158", "module_path": "qlib.contrib.data.handler", "kwargs": data_handler_config},
            "segments": {"train": ("2008-01-01", "2014-12-31"), "valid": ("2015-01-01", "2016-12-31"), "test": ("2017-01-01", "2020-08-01")},
        },
    },
}

# 初始化模型和数据集
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])



[40568:MainThread](2025-07-22 17:20:20,629) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[40568:MainThread](2025-07-22 17:20:20,631) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[40568:MainThread](2025-07-22 17:20:20,632) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/Administrator/.qlib/qlib_data/cn_data')}
[40568:MainThread](2025-07-22 17:21:37,348) INFO - qlib.timer - [log.py:127] - Time cost: 76.715s | Loading data Done
[40568:MainThread](2025-07-22 17:21:37,804) INFO - qlib.timer - [log.py:127] - Time cost: 0.153s | DropnaLabel Done
[40568:MainThread](2025-07-22 17:21:39,314) INFO - qlib.timer - [log.py:127] - Time cost: 1.509s | CSZScoreNorm Done
[40568:MainThread](2025-07-22 17:21:39,330) INFO - qlib.timer - [log.py:127] - Time cost: 1.981s | fit & process data Done
[40568:MainThread](2025-07-22 17:21:39,332) INFO - qlib.timer - [log.py:127] - Ti

In [19]:

import os  # 导入 os 模块

# --------------------------
# 关键：手动创建 mlruns 文件夹
# --------------------------
# 在调用 MLflow 前，确保 mlruns 目录存在
if not os.path.exists("mlruns"):
    os.makedirs("mlruns")

# --------------------------
# 关键：用MLflow直接创建实验
# --------------------------
experiment_name = "train_model"

# 用 set_experiment，如果不存在会自动创建
mlflow.set_experiment(experiment_name)

# 启动MLflow运行
with mlflow.start_run() as run:  # 不需要 experiment_id
    R.set_experiment_id(run.info.experiment_id)  # 从 run 里获取
    R.set_recorder_id(run.info.run_id)

    model.fit(dataset)

    # 保存模型
    mlflow.sklearn.log_model(model.model, "lgb_model")

    print(f"训练完成！")
    print(f"实验ID: {run.info.experiment_id}")
    print(f"记录器ID: {run.info.run_id}")


Exception: Invalid parent directory 'C:\Users\Administrator\Desktop\qlib-main (3)\mlruns\.trash'

In [25]:
import os
import shutil

# 清理所有 MLflow 相关目录
mlflow_dirs = ['mlruns', '.trash']
for dir_name in mlflow_dirs:
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)

In [26]:
# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

Exception: 'C:\\Users\Administrator\Desktop\qlib-main (3)\mlruns' does not exist.